<a href="https://colab.research.google.com/github/Anya232536/study_hse/blob/main/BigData_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install stuff

In [ ]:
! pip3 install pyspark pandas scikit-learn

In [ ]:
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import month, dayofmonth, dayofweek, hour
from pyspark.sql.functions import col, when, sum as _sum, avg as _avg, max as _max, expr, unix_timestamp, desc, row_number, length, size, split, count, datediff, lag, explode
from pyspark.sql.window import Window
from pyspark.ml.feature import StandardScaler, OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.functions import vector_to_array
from pyspark.ml.feature import Tokenizer, StopWordsRemover


# setup pyspark

In [ ]:
import pyspark.sql
from pyspark.sql import functions as sf
import pyspark

In [ ]:
sparkConf = pyspark.SparkConf() \
    .set("spark.executor.memory", "512m")\
    .set("spark.driver.memory", "512m")
ss = pyspark.sql.SparkSession.builder.config(conf=sparkConf).master('local[1]').getOrCreate()

# load data

In [ ]:
!unzip hw_data.zip -d ./hw_data

Archive:  hw_data.zip
   creating: ./hw_data/channels_orc/
 extracting: ./hw_data/channels_orc/.part-00003-6b7f7a7f-ffd7-44da-bd14-b7b142917916-c000.snappy.orc.crc  
  inflating: ./hw_data/channels_orc/part-00000-6b7f7a7f-ffd7-44da-bd14-b7b142917916-c000.snappy.orc  
  inflating: ./hw_data/channels_orc/part-00002-6b7f7a7f-ffd7-44da-bd14-b7b142917916-c000.snappy.orc  
 extracting: ./hw_data/channels_orc/._SUCCESS.crc  
 extracting: ./hw_data/channels_orc/.part-00002-6b7f7a7f-ffd7-44da-bd14-b7b142917916-c000.snappy.orc.crc  
 extracting: ./hw_data/channels_orc/.part-00000-6b7f7a7f-ffd7-44da-bd14-b7b142917916-c000.snappy.orc.crc  
 extracting: ./hw_data/channels_orc/_SUCCESS  
 extracting: ./hw_data/channels_orc/.part-00001-6b7f7a7f-ffd7-44da-bd14-b7b142917916-c000.snappy.orc.crc  
  inflating: ./hw_data/channels_orc/part-00003-6b7f7a7f-ffd7-44da-bd14-b7b142917916-c000.snappy.orc  
  inflating: ./hw_data/channels_orc/part-00001-6b7f7a7f-ffd7-44da-bd14-b7b142917916-c000.snappy.orc  
   cre

Читаем и предобрабатываем данные

In [ ]:
train_data = ss.read.orc('./hw_data/posts_train.orc')
test_data = ss.read.orc('./hw_data/posts_test.orc')
channel_data = ss.read.orc('./hw_data/channels_orc')

In [ ]:
#исходно выбираю те признаки, с которыми буду работать ниже
cols = ['channel_id', 'id', 'tg_id', 'text', 'has_image', 'is_forwarded', 'date', 'views']
train_data = train_data.select(*cols)

#предобрабатываю признаки has_image и is_forwarded, перевожу из текстового формата в числовой
train_data = train_data.withColumn("has_image", when(col("has_image") == "t", 1).otherwise(0))
train_data = train_data.withColumn("is_forwarded", when(col("is_forwarded") == "t", 1).otherwise(0))

train_data.show(5)

+----------+--------+-----+--------------------+---------+------------+--------------------+-----+
|channel_id|      id|tg_id|                text|has_image|is_forwarded|                date|views|
+----------+--------+-----+--------------------+---------+------------+--------------------+-----+
|       221|29741094| 7182|МК, а это вот про...|        0|           0|2018-11-03 13:05:...|15368|
|       221|46751120| 7388|Хочу пошутить, чт...|        0|           0|2018-11-26 16:15:...|12448|
|       221|32631368| 5512|Уточню: Котляр бы...|        0|           0|2018-05-16 16:03:...|69984|
|       221|46751758| 7173|Да не на два лаге...|        0|           0|2018-11-02 13:45:...|10241|
|       221|33073441| 3185|А что касается Уд...|        0|           0|2017-08-10 13:44:...|46847|
+----------+--------+-----+--------------------+---------+------------+--------------------+-----+
only showing top 5 rows



In [ ]:
#аналогично предобработке обучающей выборки выше
cols = ['channel_id', 'id', 'tg_id', 'text', 'has_image', 'is_forwarded', 'date']
test_data = test_data.select(*cols)

test_data = test_data.withColumn("has_image", when(col("has_image") == "t", 1).otherwise(0))
test_data = test_data.withColumn("is_forwarded", when(col("is_forwarded") == "t", 1).otherwise(0))

test_data.show(5)

+----------+--------+-----+--------------------+---------+------------+--------------------+
|channel_id|      id|tg_id|                text|has_image|is_forwarded|                date|
+----------+--------+-----+--------------------+---------+------------+--------------------+
|      4127|53225416| 2137|История про песик...|        0|           0|2019-01-14 15:42:...|
|      4127|53225442| 2135|Наверное, вы заме...|        0|           0|2019-01-10 11:35:...|
|     34317|53042118|11612|ર્ષમાં ફેલાઈ છે જ...|        0|           0|2019-01-05 13:40:...|
|     34317|53042119|39544|ર્ષમાં ફેલાઈ છે જ...|        0|           1|2019-01-05 13:40:...|
|     34317|53042120|11611|*🎯Pls don't copy...|        0|           0|2019-01-05 13:40:...|
+----------+--------+-----+--------------------+---------+------------+--------------------+
only showing top 5 rows



Делаю фичи из метаданных канала, возьму столбцы post_count и user_count без изменений, считаю, что это важные фичи, характеризующие сами каналы, на которых выкладываются посты.

Помимо данных фич также возьму длину заголовка поста, длину ника канала, а также создам бинарную фичу есть у канала описание или нет.


In [ ]:
channel_data.show(5)

+--------------------+----------+--------------------+----------------+----------+----------+--------------------+--------------------+----------+----------+
|         description|is_private|         last_parsed|            name|post_count|     tg_id|               title|             updated|user_count|channel_id|
+--------------------+----------+--------------------+----------------+----------+----------+--------------------+--------------------+----------+----------+
|                    |     false|2019-01-26 16:53:...|   MoeinZchannel|     708.0|1002972402|             Moein Z|2019-01-26 16:53:...|   62411.0|      7910|
|Вокруг столько ме...|     false|2019-02-12 00:39:...|  merzotachannel|    1027.0|1336284461|            Мерзость|2019-02-12 00:39:...|   12982.0|     14121|
|🗣እኛስ የተሰቀለውን ክርስ...|     false|2019-02-10 06:57:...|christian_mezmur|    1168.0|1136987361|Christian Mezmur ...|2019-02-10 06:57:...|   21704.0|     17375|
|     Chiroyli_qomatt|     false|2018-12-08 00:54:...

In [ ]:
channel_data = channel_data.withColumn('length_title', length('title')) #длина заголовка
channel_data = channel_data.withColumn('length_name', length('name')) #длина названия
# channel_data = channel_data.withColumn('num_words_title', size(split(col('title'), r'\s+'))) #количество слов в заголовке, хотела добавить, но тогда не хвататет памяти для сохранения как csv
channel_data = channel_data.withColumn("has_description", when((col("description") != "") & (col("description").isNotNull()), 1).otherwise(0)) #бинарная фича - наличие описания

#средняя длина заголовков постов и среднее количество слов в заголовке постов в каналах
avg_channel = channel_data.groupBy("channel_id").agg(
    _avg("length_title").alias("avg_length_title"))
    # _avg("num_words_title").alias("avg_num_words_title"))

channel_data = channel_data.join(avg_channel.select("channel_id", 'avg_length_title'), on="channel_id", how="left")

cols = ['channel_id', 'avg_length_title', 'user_count', 'post_count', 'length_name', 'has_description']

channel_data = channel_data.select(*cols)
channel_data.show(5)

+----------+----------------+----------+----------+-----------+---------------+
|channel_id|avg_length_title|user_count|post_count|length_name|has_description|
+----------+----------------+----------+----------+-----------+---------------+
|      7910|             7.0|   62411.0|     708.0|         13|              0|
|     14121|             8.0|   12982.0|    1027.0|         14|              1|
|     17375|            30.0|   21704.0|    1168.0|         16|              1|
|     49797|            16.0|    6516.0|     775.0|         16|              1|
|     33685|            28.0|   18995.0|    1468.0|         13|              0|
+----------+----------------+----------+----------+-----------+---------------+
only showing top 5 rows



In [ ]:
train_data = train_data.join(channel_data.select("channel_id", 'user_count', 'post_count', 'avg_length_title', 'length_name', 'has_description'), on="channel_id", how="left")

test_data = test_data.join(channel_data.select("channel_id", 'user_count', 'post_count', 'avg_length_title', 'length_name', 'has_description'), on="channel_id", how="left")


Делаю фичи из времени публикации, извлекаю месяц, день, время и день недели. Кажется, что например, вечером и в выходные количество просмотров выше.

In [ ]:
train_data = train_data.withColumn('date', regexp_replace('date', r'\+00$', '')) #проверила все заканчиваются на +00, уберу это тогда
train_data = train_data.withColumn('date', to_timestamp('date', "yyyy-MM-dd HH:mm:ss"))
train_data = train_data.withColumn('month', month('date')).withColumn('day', dayofmonth('date')) \
       .withColumn('weekday', dayofweek('date')) \
       .withColumn('hour', hour('date'))

train_data.show(5)

+----------+--------+-----+--------------------+---------+------------+-------------------+-----+----------+----------+----------------+-----------+---------------+-----+---+-------+----+
|channel_id|      id|tg_id|                text|has_image|is_forwarded|               date|views|user_count|post_count|avg_length_title|length_name|has_description|month|day|weekday|hour|
+----------+--------+-----+--------------------+---------+------------+-------------------+-----+----------+----------+----------------+-----------+---------------+-----+---+-------+----+
|       221|29741094| 7182|МК, а это вот про...|        0|           0|2018-11-03 13:05:21|15368|    8732.0|    1015.0|            12.0|         12|              1|   11|  3|      7|  13|
|       221|46751120| 7388|Хочу пошутить, чт...|        0|           0|2018-11-26 16:15:00|12448|    8732.0|    1015.0|            12.0|         12|              1|   11| 26|      2|  16|
|       221|32631368| 5512|Уточню: Котляр бы...|        0|  

In [ ]:
test_data = test_data.withColumn('date', regexp_replace('date', r'\+00$', '')) #проверила все заканчиваются на +00, уберу это тогда
test_data = test_data.withColumn('date', to_timestamp('date', "yyyy-MM-dd HH:mm:ss"))
test_data = test_data.withColumn('month', month('date')).withColumn('day', dayofmonth('date')) \
       .withColumn('weekday', dayofweek('date')) \
       .withColumn('hour', hour('date'))

test_data.show(5)

+----------+--------+-----+--------------------+---------+------------+-------------------+----------+----------+----------------+-----------+---------------+-----+---+-------+----+
|channel_id|      id|tg_id|                text|has_image|is_forwarded|               date|user_count|post_count|avg_length_title|length_name|has_description|month|day|weekday|hour|
+----------+--------+-----+--------------------+---------+------------+-------------------+----------+----------+----------------+-----------+---------------+-----+---+-------+----+
|      4127|53225416| 2137|История про песик...|        0|           0|2019-01-14 15:42:16|    5399.0|    1974.0|            14.0|         14|              1|    1| 14|      2|  15|
|      4127|53225442| 2135|Наверное, вы заме...|        0|           0|2019-01-10 11:35:21|    5399.0|    1974.0|            14.0|         14|              1|    1| 10|      5|  11|
|     34317|53042118|11612|ર્ષમાં ફેલાઈ છે જ...|        0|           0|2019-01-05 13:40:24

Извлекаю фичи из текста, добавляю такие признаки как количество слов, количество хэштегов, упоминаний, вопросов, ссылок

In [ ]:
train_data = train_data.fillna({'text': ''})

# train_data = train_data.withColumn('length_text', length('text'))  # количество символов, хотела добавить, но тогда не хвататет памяти для сохранения как csv
train_data = train_data.withColumn('num_links', size(expr("regexp_extract_all(text, 'https?://[^\\s]+', 0)"))) # количество ссылок
train_data = train_data.withColumn('num_hashtags', size(expr("regexp_extract_all(text, '#\\w+', 0)"))) # количество хэштегов
train_data = train_data.withColumn('num_mentions', length(regexp_replace('text', '[^@]', ''))) # количество упоминаний
train_data = train_data.withColumn('num_questions', length(regexp_replace('text', '[^?]', ''))) #количество вопросов


In [ ]:
tokenizer = Tokenizer(inputCol="text", outputCol="tokens")
train_data = tokenizer.transform(train_data)

train_data = train_data.withColumn("num_words", size("tokens")) # количество слов
train_data = train_data.drop("tokens", 'text')
train_data.show()

+----------+--------+-----+---------+------------+-------------------+-----+----------+----------+----------------+-----------+---------------+-----+---+-------+----+---------+------------+------------+-------------+---------+
|channel_id|      id|tg_id|has_image|is_forwarded|               date|views|user_count|post_count|avg_length_title|length_name|has_description|month|day|weekday|hour|num_links|num_hashtags|num_mentions|num_questions|num_words|
+----------+--------+-----+---------+------------+-------------------+-----+----------+----------+----------------+-----------+---------------+-----+---+-------+----+---------+------------+------------+-------------+---------+
|       221|29741094| 7182|        0|           0|2018-11-03 13:05:21|15368|    8732.0|    1015.0|            12.0|         12|              1|   11|  3|      7|  13|        1|           0|           0|            4|        7|
|       221|46751120| 7388|        0|           0|2018-11-26 16:15:00|12448|    8732.0|    1

In [ ]:
test_data = test_data.fillna({'text': ''})

# test_data = test_data.withColumn('length_text', length('text'))  # количество символов, хотела добавить, но тогда не хвататет памяти для сохранения как csv
test_data = test_data.withColumn('num_links', size(expr("regexp_extract_all(text, 'https?://[^\\s]+', 0)"))) # количество ссылок
test_data = test_data.withColumn('num_hashtags', size(expr("regexp_extract_all(text, '#\\w+', 0)"))) # количество хэштегов
test_data = test_data.withColumn('num_mentions', length(regexp_replace('text', '[^@]', ''))) # количество упоминаний
test_data = test_data.withColumn('num_questions', length(regexp_replace('text', '[^?]', ''))) #количество вопросов


In [ ]:
tokenizer = Tokenizer(inputCol="text", outputCol="tokens")
test_data = tokenizer.transform(test_data)

test_data = test_data.withColumn("num_words", size("tokens")) # количество слов
test_data = test_data.drop("tokens", "text")
test_data.show()

+----------+--------+-----+---------+------------+-------------------+----------+----------+----------------+-----------+---------------+-----+---+-------+----+---------+------------+------------+-------------+---------+
|channel_id|      id|tg_id|has_image|is_forwarded|               date|user_count|post_count|avg_length_title|length_name|has_description|month|day|weekday|hour|num_links|num_hashtags|num_mentions|num_questions|num_words|
+----------+--------+-----+---------+------------+-------------------+----------+----------+----------------+-----------+---------------+-----+---+-------+----+---------+------------+------------+-------------+---------+
|      4127|53225416| 2137|        0|           0|2019-01-14 15:42:16|    5399.0|    1974.0|            14.0|         14|              1|    1| 14|      2|  15|        1|           0|           0|            1|      145|
|      4127|53225442| 2135|        0|           0|2019-01-10 11:35:21|    5399.0|    1974.0|            14.0|       

Извлекаю оконные фичи:

- среднее число просмотров по всем предыдущим постам в канале: то есть беру все посты до текущего поста и извлекаю среднее число просмотров на канале. Для тестовой выборки - среднее число просмотров на канале по всей обучающей выборке.

- аналогично извлекаю фичи не по всем просмотрам на канале, а за более короткие периоды: среднее число просмотров за неделю и за месяц, кажется, что данные фичи показывают краткосрочный и среднесрочный тренд. При этом для тестовой выборки я также буду забивать тест последними рассчитанными статистиками по обучающей выборке. То есть для всех тестовых данных признаки "среднее число просмотров за 7 дней" и "среднее число просмотров за 30 дней" будут определены как среднее число просмотров за данные периоды, рассчитанные для каждого канала по последним данным из обучающейся выборки.

- также беру время прошедшее с предыдущей публикации для каждого поста, насколько я знаю, для большого числа просмотров важно регулярно выкладывать посты. Разницу буду брать в часах, в тестовой выборке для первой даты эта фича будет найдена как разница между последний датой из трэйна и первой из теста для каждого канала.

In [ ]:
windowSpec = Window.partitionBy("channel_id").orderBy("tg_id").rowsBetween(Window.unboundedPreceding, -1)

# Среднее число просмотров по предыдущим постам в канале
train_data = train_data.withColumn("avg_views", _avg("views").over(windowSpec))

# Для расчёта предыдущей даты и разницы
window_lag = Window.partitionBy("channel_id").orderBy("tg_id")
train_data = train_data.withColumn("prev_date", lag("date").over(window_lag)) #дата предыдущего поста

# Сначала рассчитаем в секундах
train_data = train_data.withColumn("timestamp", unix_timestamp("date"))
train_data = train_data.withColumn("prev_timestamp", unix_timestamp("prev_date"))

# Разница в часах между постами
train_data = train_data.withColumn(
    "hours_since_prev_post",
    (col("timestamp") - col("prev_timestamp")) / 3600
)

In [ ]:
#здесь как раз рассчитываю среднее число просмотров за 7 и 30 дней

week_window = Window.partitionBy("channel_id") \
    .orderBy("timestamp") \
    .rangeBetween(-7 * 86400, -1)  # 7 дней в секундах

month_window = Window.partitionBy("channel_id") \
    .orderBy("timestamp") \
    .rangeBetween(-30 * 86400, -1)  # 30 дней в секундах

train_data = train_data.withColumn("avg_views_week", _avg("views").over(week_window))
train_data = train_data.withColumn("avg_views_month", _avg("views").over(month_window))

In [ ]:
#рассчитываю среднее число просмотров по обучающей выборки + ищу последнюю дату
avg_views_per_channel = train_data.groupBy("channel_id").agg(
    _avg("views").alias("avg_views"),
    _max("date").alias("last_train_date")
)

test_data = test_data.join(avg_views_per_channel, on="channel_id", how="left")

# Аналогично train
window_lag = Window.partitionBy("channel_id").orderBy("tg_id")
test_data = test_data.withColumn("prev_date", lag("date").over(window_lag))

# Сначала рассчитаем в секундах
test_data = test_data.withColumn("timestamp", unix_timestamp("date"))
test_data = test_data.withColumn("prev_timestamp", unix_timestamp("prev_date"))
test_data = test_data.withColumn("last_train_timestamp", unix_timestamp("last_train_date"))

# Разница в часах между постами
test_data = test_data.withColumn(
    "hours_since_prev_post",
    when(
        col("prev_timestamp").isNotNull(),
        (col("timestamp") - col("prev_timestamp")) / 3600   #вот здесь как раз учитываю, что для первой даты из теста берется разница с последней датой из трейна
    ).otherwise(
        (col("timestamp") - col("last_train_timestamp")) / 3600
    )
)

In [ ]:
#максимальная дата в секундах по каждому каналу
max_timestamps = train_data.groupBy("channel_id").agg(
    _max("timestamp").alias("max_ts")
)

train_with_max = train_data.join(max_timestamps, on="channel_id", how="left")

# последние 7 дней по в трейне
last_7d = train_with_max.filter(
    (col("timestamp") >= col("max_ts") - 7 * 86400)
)

# последние 30 дней по в трейне
last_30d = train_with_max.filter(
    (col("timestamp") >= col("max_ts") - 30 * 86400)
)

# средние просмотры за последние 7 дней
avg_7d = last_7d.groupBy("channel_id").agg(
    _avg("views").alias("avg_views_week")
)

# средние просмотры за последние 30 дней
avg_30d = last_30d.groupBy("channel_id").agg(
    _avg("views").alias("avg_views_month")
)

avg_views_recent = avg_7d.join(avg_30d, on="channel_id", how="outer")
test_data = test_data.join(avg_views_recent, on="channel_id", how="left")


In [ ]:
cols_train = ['id', 'has_image', 'is_forwarded', 'user_count', 'post_count', 'avg_length_title', 'length_name', 'has_description', 'month', 'day', 'weekday', 'hour',
              'num_links', 'num_hashtags', 'num_mentions', 'num_questions', 'num_words', 'avg_views', 'hours_since_prev_post', 'avg_views_week', 'avg_views_month', 'views']
train_data = train_data[cols_train]

In [ ]:
cols_test = ['id', 'has_image', 'is_forwarded', 'user_count', 'post_count', 'avg_length_title', 'length_name', 'has_description', 'month', 'day', 'weekday', 'hour',
              'num_links', 'num_hashtags', 'num_mentions', 'num_questions', 'num_words', 'avg_views', 'hours_since_prev_post', 'avg_views_week', 'avg_views_month']
test_data = test_data[cols_test]

In [ ]:
train_data.write.csv('train_csv', mode='overwrite', header=True)
test_data.write.csv('test_csv', mode='overwrite', header=True)

In [ ]:
import subprocess
import glob
import os
import shutil
import pandas as pd

def load_and_merge_csv(path, **kwargs):
    dfs = []
    for g in glob.glob(os.path.join(path, '*.csv')):
        dfs.append(pd.read_csv(g, **kwargs))
    res = pd.concat(dfs)
    res = res.set_index('id')
    return res

In [ ]:
trainXY = load_and_merge_csv('train_csv')
testX = load_and_merge_csv('test_csv')

In [ ]:
import numpy as np

Ycol = 'views'
trainX, trainY = trainXY.drop(Ycol, axis=1), trainXY[Ycol]
trainY = np.log(trainY + 100)

trainX['avg_views'] = np.log(trainX['avg_views']+100)
testX['avg_views'] = np.log(testX['avg_views']+100)
trainX['avg_views_month'] = np.log(trainX['avg_views_month']+100)
testX['avg_views_month'] = np.log(testX['avg_views_month']+100)
trainX['avg_views_week'] = np.log(trainX['avg_views_week']+100)
testX['avg_views_week'] = np.log(testX['avg_views_week']+100)

#testX = testX.drop(to_drop, axis=1)

In [ ]:
trainX

,has_image,is_forwarded,user_count,post_count,avg_length_title,length_name,has_description,month,day,weekday,hour,num_links,num_hashtags,num_mentions,num_questions,num_words,avg_views,hours_since_prev_post,avg_views_week,avg_views_month
id,,,,,,,,,,,,,,,,,,,,
12106335,0,0,9242.0,4380.0,11.0,9,1,11,11,7,18,1,0,0,0,1,NaN,NaN,NaN,NaN
12106334,0,0,9242.0,4380.0,11.0,9,1,11,11,7,18,1,0,0,1,1,4.962845,0.078056,4.962845,4.962845
12106333,0,0,9242.0,4380.0,11.0,9,1,11,12,1,8,1,0,0,1,1,4.945207,13.414167,4.945207,4.945207
12106332,0,0,9242.0,4380.0,11.0,9,1,11,12,1,8,1,0,0,1,9,4.927254,0.030556,4.927254,4.927254
12106331,0,0,9242.0,4380.0,11.0,9,1,11,12,1,8,1,0,0,1,7,4.910815,0.216111,4.910815,4.910815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57000107,1,0,9790.0,790.0,15.0,22,1,12,27,5,8,0,0,1,0,7,6.358318,0.003333,6.538791,6.358318
57000106,1,0,9790.0,790.0,15.0,22,1,12,27,5,8,0,0,1,0,7,6.360757,0.003333,6.540432,6.360757
57000105,1,0,9790.0,790.0,15.0,22,1,12,27,5,8,0,0,1,0,7,6.363283,0.003333,6.542300,6.363283


In [ ]:
#забиваю нулы
trainX = trainX.fillna(method='bfill')

/tmp/ipython-input-29-2774825209.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  trainX = trainX.fillna(method='bfill')


In [ ]:
testX

,has_image,is_forwarded,user_count,post_count,avg_length_title,length_name,has_description,month,day,weekday,hour,num_links,num_hashtags,num_mentions,num_questions,num_words,avg_views,hours_since_prev_post,avg_views_week,avg_views_month
id,,,,,,,,,,,,,,,,,,,,
54363782,0,0,27660.0,1397.0,13.0,8,1,1,4,6,10,0,0,0,1,205,9.359764,168.426389,10.238130,10.350641
54363781,0,0,27660.0,1397.0,13.0,8,1,1,4,6,10,0,0,0,1,60,9.359764,0.129444,10.238130,10.350641
54363780,0,0,27660.0,1397.0,13.0,8,1,1,5,7,11,0,0,0,2,179,9.359764,25.384722,10.238130,10.350641
54363777,0,0,27660.0,1397.0,13.0,8,1,1,8,3,11,0,0,0,1,61,9.359764,71.466667,10.238130,10.350641
54363774,0,0,27660.0,1397.0,13.0,8,1,1,11,6,18,0,0,0,1,368,9.359764,78.629722,10.238130,10.350641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57062229,0,0,23872.0,3340.0,11.0,10,1,2,12,3,20,4,0,0,0,140,9.481068,0.005833,9.590602,9.646854
57062228,1,0,23872.0,3340.0,11.0,10,1,2,13,4,8,4,0,0,1,148,9.481068,12.402778,9.590602,9.646854
57062227,1,0,23872.0,3340.0,11.0,10,1,2,13,4,11,2,0,0,0,47,9.481068,2.831944,9.590602,9.646854


In [ ]:
#в тестовой выборки нуллов нет
testX.isna().sum()

,0
has_image,0
is_forwarded,0
user_count,0
post_count,0
avg_length_title,0
length_name,0
has_description,0
month,0
day,0
weekday,0


# train your model and predict test

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(trainX, trainY, test_size=0.2, random_state=12)

In [ ]:
def metrics(y_test, y_test_pred):
  result = {'mape': mean_absolute_percentage_error(y_test, y_test_pred),
            'mean_absolute_error': mean_absolute_error(np.array(y_test), y_test_pred),
            'mean_squared_error': mean_squared_error(y_test, y_test_pred),
            'rmse': mean_squared_error(y_test, y_test_pred),
            'rmspe': np.sqrt(np.mean(np.square(((y_test - y_test_pred) / y_test)), axis=0))}
  print("rmse = %.3f" % result['mape'])
  print("mean_squared_error = %.3f" % result['mean_absolute_error'])
  print("rmspe = %.3f" % result['mean_squared_error'])
  print("mape = %.3f" % result['rmse'])
  print("mean_absolute_error = %.3f" % result['rmspe'])

Линейная регрессия

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_val)
metrics(y_val, y_pred)

rmse = 0.076
mean_squared_error = 0.551
rmspe = 0.850
mape = 0.850
mean_absolute_error = 0.154


Бустинг ( хотела еще добавить случайный лес, но училось долго + бейзлайн побит уже в линейной регрессии)

In [ ]:
from xgboost import XGBRegressor

xgb = XGBRegressor()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_val)
metrics(y_val, y_pred)

rmse = 0.050
mean_squared_error = 0.369
rmspe = 0.480
mape = 0.480
mean_absolute_error = 0.113


In [ ]:
prediction = xgb.predict(testX)

In [ ]:
prediction.shape

(244386,)

In [ ]:
assert prediction.shape == (244386,)

# submit

In [ ]:
! curl -H 'Cache-Control: no-cache' https://raw.githubusercontent.com/mike0sv/lsml_submit_server/2024/src/client.py -o client.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1899  100  1899    0     0   9237      0 --:--:-- --:--:-- --:--:--  9263


In [ ]:
import client

In [ ]:
client.make_eval(pd.DataFrame({'views': prediction}, index=testX.index), final=True)

Enter username:
hse-07
Enter password:
··········


{'data': {'mape': 7.533098981549275,
  'mean_absolute_error': 0.5727729409264365,
  'mean_squared_error': 0.9515377771372617,
  'rmse': 0.9754679785299268,
  'rmspe': 14.467916492532202},
 'ok': True}

In [ ]:
client.check_results()

{'2025-06-17': [{'baseline_beaten': False,
   'is_final': False,
   'metrics': {'mape': 93.91877445611847,
    'mean_absolute_error': 8.07459841845251,
    'mean_squared_error': 67.63713391390488,
    'rmse': 8.224179832293606,
    'rmspe': 93.99923209105816}},
  {'baseline_beaten': False,
   'is_final': False,
   'metrics': {'mape': 93.9295929792797,
    'mean_absolute_error': 8.075397448912213,
    'mean_squared_error': 67.6498877216629,
    'rmse': 8.224955180526086,
    'rmspe': 94.00955928482544}},
  {'baseline_beaten': False,
   'is_final': False,
   'metrics': {'mape': 93.92466813974943,
    'mean_absolute_error': 8.075250884964628,
    'mean_squared_error': 67.65047607564259,
    'rmse': 8.224990946842592,
    'rmspe': 94.00510520749238}},
  {'baseline_beaten': False,
   'is_final': False,
   'metrics': {'mape': 93.9267759045814,
    'mean_absolute_error': 8.07517955301921,
    'mean_squared_error': 67.64668623388692,
    'rmse': 8.22476055784525,
    'rmspe': 94.00684543628469